In [25]:
import math
import numpy as np
import glob
import os
import torch

data_folder = os.path.join('Synthetic_v3_InstanceSegmentation')
files = sorted(glob.glob(data_folder + '/*.txt'))
numclass = 15
semanticIDs = []
for i in range(numclass):
    semanticIDs.append(i)

class_numpoint_mean_dict = {}
class_radius_mean = {}
for semanticID in semanticIDs:
    class_numpoint_mean_dict[semanticID] = []
    class_radius_mean[semanticID] = []
num_points_semantic = np.array([0 for i in range(numclass)])

for file in files:
    data_buff = np.loadtxt(file,delimiter=",")
    coords, colors, sem_labels, instance_labels =data_buff[:,:3],data_buff[:,3:6],data_buff[:,-2],data_buff[:,-1]
    points = np.concatenate((coords,colors,np.expand_dims(sem_labels.astype(int),axis=1),np.expand_dims(instance_labels.astype(int),axis=1)),axis=1)
    for semanticID in semanticIDs:
        singleSemantic = points[np.where(points[:,6]==semanticID)]
        uniqueInstances, counts = np.unique(singleSemantic[:,7], return_counts=True)
        for count in counts:
            class_numpoint_mean_dict[semanticID].append(count)
        allRadius = []
        for uniqueInstance in uniqueInstances:
            eachInstance = singleSemantic[np.where(singleSemantic[:,7]==uniqueInstance)]
            radius = (np.max(eachInstance,axis=0) - np.min(eachInstance,axis=0))/2
            radius = math.sqrt(radius[0]**2 + radius[1]**2 + radius[2]**2)
            class_radius_mean[semanticID].append(radius)

    uniqueSemantic,semanticCount = np.unique(points[:,6],return_counts=True)
    uniqueSemanticCount = np.array([0 for i in range(numclass)])
    uniqueSemantic = uniqueSemantic.astype(int)
    indexOf100 = np.where(uniqueSemantic == -100)
    semanticCount = np.delete(semanticCount, indexOf100)
    uniqueSemantic = np.delete(uniqueSemantic, indexOf100)
    uniqueSemanticCount[uniqueSemantic] = semanticCount
    num_points_semantic += uniqueSemanticCount

class_numpoint_mean_list = []
class_radius_mean_list = []


In [26]:
for semanticID in semanticIDs:
    class_numpoint_mean_list.append(sum(class_numpoint_mean_dict[semanticID])*1.0/len(class_numpoint_mean_dict[semanticID]))
    class_radius_mean_list.append(sum(class_radius_mean[semanticID])/len(class_radius_mean[semanticID]))

print ("Using the printed list in hierarchical_aggregation.cpp for class_numpoint_mean_dict: ")
print ([1.0]+[float("{0:0.0f}".format(i)) for i in class_numpoint_mean_list][1:], sep=',')
print ("Using the printed list in hierarchical_aggregation.cu for class_radius_mean: ")
print ([1.0]+[float("{0:0.2f}".format(i)) for i in class_radius_mean_list][1:], sep='')

### make ground to 1 the make building to 1
maxSemantic = np.max(num_points_semantic)
num_points_semantic = maxSemantic/num_points_semantic
num_points_semantic = num_points_semantic/num_points_semantic[1]
print ("Using the printed list in hais_run_stpls3d.yaml for class_weight")
print ([1.0,1.0]+[float("{0:0.2f}".format(i)) for i in num_points_semantic][2:], sep='')

Using the printed list in hierarchical_aggregation.cpp for class_numpoint_mean_dict: 
[1.0, 10355.0, 59.0, 124.0, 1334.0, 161.0, 425.0, 1081.0, 450.0, 26.0, 43.0, 61.0, 40.0, 109.0, 1308.0]
Using the printed list in hierarchical_aggregation.cu for class_radius_mean: 
[1.0, 19.79, 3.02, 3.36, 8.45, 2.99, 4.36, 10.56, 4.91, 3.69, 1.64, 5.83, 3.22, 2.5, 16.67]
Using the printed list in hais_run_stpls3d.yaml for class_weight
[1.0, 1.0, 300.43, 160.92, 1.76, 26.16, 34.13, 156.68, 172.63, 392.34, 106.22, 88.87, 297.43, 72.74, 21.56]


# Berbeda masalah. 
Using the printed list in hierarchical_aggregation.cpp for class_numpoint_mean_dict: 
[1.0, 10355.0, 59.0, 124.0, 1334.0, 161.0, 425.0, 1081.0, 450.0, 26.0, 43.0, 61.0, 40.0, 109.0, 1308.0]
Using the printed list in hierarchical_aggregation.cu for class_radius_mean: 
[1.0, 19.79, 3.02, 3.36, 8.45, 2.99, 4.36, 10.56, 4.91, 3.69, 1.64, 5.83, 3.22, 2.5, 16.67]
Using the printed list in hais_run_stpls3d.yaml for class_weight
[1.0, 1.0, 300.43, 160.92, 1.76, 26.16, 34.13, 156.68, 172.63, 392.34, 106.22, 88.87, 297.43, 72.74, 21.56]

In [22]:
data_test = np.loadtxt("Synthetic_v3_InstanceSegmentation/1_points_GTv3.txt",delimiter=",")

yang sali
Using the printed list in hierarchical_aggregation.cpp for class_numpoint_mean_dict: 
[1.0, 4121.0, 53.0, 115.0, 1013.0, 141.0, 363.0, 777.0, 360.0, 23.0, 40.0, 56.0, 35.0, 100.0, 605.0]
Using the printed list in hierarchical_aggregation.cu for class_radius_mean: 
[1.0, 12.87, 2.02, 2.67, 7.46, 2.73, 3.88, 7.84, 4.13, 2.22, 1.5, 5.2, 2.47, 2.24, 10.28]
Using the printed list in hais_run_stpls3d.yaml for class_weight
[1.0, 1.0, 50.71, 27.48, 1.39, 20.51, 25.37, 45.42, 60.12, 79.72, 85.26, 64.34, 73.18, 13.35, 17.37]

In [8]:
class_numpoint_mean_dict["0"]*1.0/len(class_numpoint_mean_dict["0"])

KeyError: '0'

In [17]:
sum(class_numpoint_mean_dict[1])/len(class_numpoint_mean_dict[1])

4105.001702946838

10408